# Clustering Crypto

In [155]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [156]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv('crypto_data.csv',index_col=0)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [157]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df['IsTrading']==True]

In [158]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns=['IsTrading'])

In [159]:
# Remove rows that have at least 1 null value.

crypto_df = crypto_df.dropna()

In [160]:
# Keep the rows where coins are mined.

crypto_df= crypto_df[crypto_df['TotalCoinsMined']>0]


In [161]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_name_df = crypto_df['CoinName'].to_frame()

crypto_name_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [162]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns=['CoinName'])
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [163]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df[['Algorithm', 'ProofType']])
X = crypto_df[['TotalCoinsMined','TotalCoinSupply']].join(X)
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [164]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)

In [165]:
X_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [166]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X_scaled)

In [167]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(crypto_pca,columns=['PC 1','PC 2','PC 3'],index=crypto_name_df.index)
pcs_df

,PC 1,PC 2,PC 3
42,-0.309791,1.092447,-0.428064
404,-0.293015,1.092827,-0.428052
1337,2.293608,1.703303,-0.529703
BTC,-0.165301,-1.341114,0.161799
ETH,-0.140103,-2.029137,0.296862
...,...,...,...
ZEPH,2.373630,0.688873,0.079640
GAP,-0.307825,1.092352,-0.428057
BDX,0.300707,-2.363656,0.321599
ZEN,-0.112227,-2.047353,0.284675


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [168]:
# Create an elbow curve to find the best value for K.

# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [169]:
# Initialize the K-Means model.
km_model = KMeans(n_clusters=4)

# Fit the model

km_model.fit(pcs_df)
# Predict clusters

predictions = km_model.predict(pcs_df)

In [170]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.

cluster_df = crypto_df.join(pcs_df)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 

cluster_df = cluster_df.join(crypto_name_df)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
cluster_df['Class'] = predictions

# Print the shape of the clustered_df
print(cluster_df.shape)
cluster_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.309791,1.092447,-0.428064,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.293015,1.092827,-0.428052,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.293608,1.703303,-0.529703,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.165301,-1.341114,0.161799,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.140103,-2.029137,0.296862,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.140407,-1.089258,0.026104,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.400358,1.232906,-0.469838,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.176674,-2.242463,0.293177,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.138536,-2.029220,0.296868,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.112226,-2.047353,0.284675,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [171]:
# Creating a 3D-Scatter with the PCA data and the clusters

fig = px.scatter_3d(
    cluster_df,
    x="PC 1", 
    y="PC 2", 
    z="PC 3", 
    hover_name='CoinName',
    hover_data=['Algorithm'],
    color="Class",
    symbol='Class',
    width=800,

    )
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [172]:
# Create a table with tradable cryptocurrencies.
import hvplot.pandas
viewer = cluster_df.hvplot.table(columns=['CoinName','Algorithm', 'ProofType','TotalCoinSupply', 'TotalCoinsMined','Class'],width=400)
hvplot.show(viewer)

Launching server at http://localhost:59950


In [173]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {cluster_df['CoinName'].count()} tradeable cryptocurrencies")

There are 532 tradeable cryptocurrencies


In [174]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.

cluster_df[['TotalCoinSupply','TotalCoinsMined']] = MinMaxScaler().fit_transform(cluster_df[['TotalCoinSupply','TotalCoinsMined']])
cluster_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,0.000000e+00,4.200000e-11,-0.309791,1.092447,-0.428064,42 Coin,1
404,Scrypt,PoW/PoS,1.065855e-03,5.320000e-04,-0.293015,1.092827,-0.428052,404Coin,1
1337,X13,PoW/PoS,2.957551e-02,3.141593e-01,2.293608,1.703303,-0.529703,EliteCoin,1
BTC,SHA-256,PoW,1.810842e-05,2.100000e-05,-0.165301,-1.341114,0.161799,Bitcoin,0
ETH,Ethash,PoW,1.087731e-04,0.000000e+00,-0.140103,-2.029137,0.296862,Ethereum,0
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.020225e-03,2.000000e-03,2.373630,0.688873,0.079640,ZEPHYR,1
GAP,Scrypt,PoW/PoS,1.508199e-05,2.500000e-04,-0.307825,1.092352,-0.428057,Gapcoin,1
BDX,CryptoNight,PoW,9.901351e-04,1.400223e-03,0.300707,-2.363656,0.321599,Beldex,0
ZEN,Equihash,PoW,7.370282e-06,2.100000e-05,-0.112227,-2.047353,0.284675,Horizen,0


In [175]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.

totals_df = cluster_df[['TotalCoinSupply','TotalCoinsMined']]

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.

plot_df = totals_df.join(crypto_name_df)

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 

plot_df['Class'] = cluster_df['Class']

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,1
404,5.320000e-04,0.001066,404Coin,1
1337,3.141593e-01,0.029576,EliteCoin,1
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,1
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [176]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".

hvplot.show(cluster_df.hvplot.scatter(x='TotalCoinsMined', y='TotalCoinSupply', by='Class', size=10, hover_cols=['CoinName']))

Launching server at http://localhost:59979
